In [3]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
import json

# Step 1: Load and Preprocess Data

# Load training data
train_data_path = "/Users/borasacir/Documents/projects/CS-412-Project/project-deliverables/training-dataset.jsonl.gz"
train_classification_path = "/Users/borasacir/Documents/projects/CS-412-Project/user-annotations.csv"
extratrain_classification_path = "/Users/borasacir/Documents/projects/CS-412-Project/project-deliverables/train-classification.csv"


In [11]:
def preprocess_text(text):
    text = text.casefold()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'[^a-z\u00c0-\u017f\s#@]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Load and preprocess training data
train_data = pd.read_json(train_data_path, lines=True)
train_data.rename(columns={"Unknown: 0": "user_id"}, inplace=True)

# Ensure posts are parsed correctly as lists of dictionaries
def preprocess_posts(posts):
    """
    Preprocess the captions in the posts field.
    """
    processed_posts = []
    for post in posts:
        caption = post.get('caption', '')
        if caption:  # Only preprocess if caption exists
            processed_caption = preprocess_text(caption)
            processed_posts.append(processed_caption)
    return ' '.join(processed_posts)  # Combine processed captions into a single string

# Apply preprocessing directly to the posts column
train_data['processed_posts'] = train_data['posts'].apply(preprocess_posts)

# Check the result
print(train_data[['processed_posts']].head())


                                     processed_posts
0  cumhuriyetimizin yılı kutlu olsun oriflame duo...
1  bu diyaloğun yaşanmadığı bir online toplantı o...
2  bugün bir fincan köpüklü türk kahvesiyle taçla...
3  saygı ve özlemle #atatürk #kasım #kasim #vimer...
4  başöğretmenimiz gazi mustafa kemal atatürkün ı...


In [12]:
# Step 2: Load User Annotations and Merge
user_annotations = pd.read_csv(train_classification_path)
extra_annotations = pd.read_csv(extratrain_classification_path)
annotations = pd.concat([user_annotations, extra_annotations], ignore_index=True)
annotations.rename(columns={"accountType": "target"}, inplace=True)

# Merge the annotations with training data
train_data = train_data.merge(annotations, on="user_id")

# Check merged dataframe
print("Merged training data sample:")
print(train_data.head())

KeyError: 'user_id'